In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kProximity)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6006...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [35]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9]
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()

In [36]:
regions = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

for i, s in enumerate(seed_points):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if False:
        #starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    print(f"{r.A()}t <= {r.b()}")
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -1.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -1.00000000e+00]
 [ 1.55625494e-08  4.95489590e-08  1.00000000e+00]
 [-9.01101183e-01  4.33608877e-01  4.99181102e-07]
 [-9.99938968e-01 -1.10480652e-02  2.23596261e-06]
 [ 7.57688004e-08  8.90282532e-08  1.00000000e+00]
 [-8.96784915e-01  4.42466740e-01 -6.35603468e-07]
 [ 1.08627953e-06 -4.68463205e-08  1.00000000e+00]
 [-8.94101373e-01  4.47864638e-01  1.04631744e-06]
 [ 3.31132200e-01 -1.66418629e-01 -9.28792929e-01]
 [ 2.73720612e-01 -1.38721523e-01 -9.51752786e-01]
 [ 3.26714414e-01 -1.65897897e-01 -9.30449128e-01]
 [ 3.67521492e-01 -1.87778998e-01 -9.10860582e-01]
 [ 8.41197495e-01 -4.03003561e-01  3.60520325e-01]]t <= [1.13833271 1.55740772 1.13833271 1.13833271 1.55740772 1.13833271
 0.2515504

In [37]:
if do_viz:
    visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

In [38]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

edited_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 5.0
binary_search_options.epsilon_min = -.12
binary_search_options.max_iters = 5
binary_search_options.search_d = True
bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-6
bilinear_alternation_option.polytope_backoff_scale = 1e-6


solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

def iris_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,))) 
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    C = ellipse.A()
    radii2, R  =  np.linalg.eig(C.T @ C)
    radii = np.sqrt(radii2)
    binary_search_options.epsilon_min = FindEpsilonLower(limits_t[0], limits_t[1], region.A(), region.b())
    print("[IRIS +  CERT]: Eps min", binary_search_options.epsilon_min)
    print("[IRIS +  CERT]: Region found, vol = ", ellipse.Volume(), ", halfaxis: ", radii)
    b_feasible = cspace_free_region.CspacePolytopeBinarySearch(
                 iris_options.q_star, filtered_collision_pairs, region.A(), region.b(),
                 binary_search_options, solver_options)
    
#     C_final, d_final, P_final, q_final = \
#         cspace_free_region.CspacePolytopeBilinearAlternation(
#             iris_options.q_star, filtered_collision_pairs, region.A(), b_feasible,
#            bilinear_alternation_option, solver_options)
#     C_final = np.vstack([C_final, P_joint_limits.A()])
#     d_final = np.concatenate([d_final, P_joint_limits.b()])
#     editted_regions.append(HPolyhedron(C_final, d_final))
    return region, HPolyhedron(region.A(), b_feasible) 

In [39]:
#refine with CPP code:
cpp_editted_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_handle(s, region_init)
    cpp_editted_regions.append(region)

[2022-01-28 10:04:39.526] [console] [info] Solver time 1.2997000217437744
[2022-01-28 10:04:42.106] [console] [info] Solver time 1.0446739196777344


[IRIS +  CERT]: Eps min -0.2652286679481278
[IRIS +  CERT]: Region found, vol =  0.6819787390374958 , halfaxis:  [4.86595118 2.17555524 0.58020292]


[2022-01-28 10:04:44.651] [console] [info] Solver time 1.228912115097046
[2022-01-28 10:04:44.692] [console] [info] epsilon=2.3673856660259363 is infeasible
[2022-01-28 10:04:48.317] [console] [info] Solver time 1.863494873046875
[2022-01-28 10:04:48.361] [console] [info] epsilon=1.0510784990389042 is infeasible
[2022-01-28 10:04:53.483] [console] [info] Solver time 2.682873010635376
[2022-01-28 10:04:53.530] [console] [info] epsilon=0.39292491554538816 is infeasible
[2022-01-28 10:04:58.577] [console] [info] Solver time 2.477357864379883
[2022-01-28 10:04:58.642] [console] [info] epsilon=0.06384812379863017 is infeasible
[2022-01-28 10:05:02.844] [console] [info] Solver time 1.9119389057159424
[2022-01-28 10:05:02.894] [console] [info] epsilon=-0.10069027207474883 is infeasible


[IRIS +  CERT]: Eps min -0.29420562277149487
[IRIS +  CERT]: Region found, vol =  0.5535987103063078 , halfaxis:  [4.69487125 2.58579857 0.62326848]


[2022-01-28 10:05:08.510] [console] [info] Solver time 1.2431840896606445
[2022-01-28 10:05:11.020] [console] [info] Solver time 0.7933089733123779
[2022-01-28 10:05:14.021] [console] [info] Solver time 1.4338359832763672
[2022-01-28 10:05:14.066] [console] [info] epsilon=2.3528971886142527 is infeasible
[2022-01-28 10:05:18.228] [console] [info] Solver time 2.0333070755004883
[2022-01-28 10:05:18.281] [console] [info] epsilon=1.029345782921379 is infeasible
[2022-01-28 10:05:25.007] [console] [info] Solver time 3.162363052368164
[2022-01-28 10:05:25.068] [console] [info] epsilon=0.3675700800749421 is infeasible
[2022-01-28 10:05:31.054] [console] [info] Solver time 2.8076210021972656
[2022-01-28 10:05:31.119] [console] [info] epsilon=0.0366822286517236 is infeasible
[2022-01-28 10:05:38.635] [console] [info] Solver time 1.8688910007476807
[2022-01-28 10:05:38.743] [console] [info] epsilon=-0.12876169705988563 is feasible
[2022-01-28 10:05:38.745] [console] [info] reset eps_min=0.0, ep

...

[2022-01-28 10:05:42.825] [console] [info] Solver time 1.212691068649292
[2022-01-28 10:05:45.236] [console] [info] Solver time 0.7653758525848389
[2022-01-28 10:05:48.017] [console] [info] Solver time 1.240004062652588
[2022-01-28 10:05:48.079] [console] [info] epsilon=2.405601513254188 is infeasible
[2022-01-28 10:05:52.281] [console] [info] Solver time 1.9871110916137695
[2022-01-28 10:05:52.335] [console] [info] epsilon=1.1084022698812819 is infeasible
[2022-01-28 10:05:59.424] [console] [info] Solver time 3.7420270442962646
[2022-01-28 10:05:59.485] [console] [info] epsilon=0.45980264819482886 is infeasible
[2022-01-28 10:06:06.328] [console] [info] Solver time 3.4925050735473633
[2022-01-28 10:06:06.405] [console] [info] epsilon=0.13550283735160235 is infeasible
[2022-01-28 10:06:11.982] [console] [info] Solver time 2.3900768756866455
[2022-01-28 10:06:12.042] [console] [info] epsilon=-0.02664706807001091 is infeasible
[2022-01-28 10:06:16.107] [console] [info] Solver time 1.2343

[IRIS +  CERT]: Eps min -0.3723186551818972
[IRIS +  CERT]: Region found, vol =  0.43219436206092116 , halfaxis:  [3.9707394  1.96475249 1.24231085]


[2022-01-28 10:06:18.854] [console] [info] Solver time 0.821234941482544
[2022-01-28 10:06:21.850] [console] [info] Solver time 1.261733055114746
[2022-01-28 10:06:21.904] [console] [info] epsilon=2.3138406724090514 is infeasible
[2022-01-28 10:06:27.076] [console] [info] Solver time 2.3649940490722656
[2022-01-28 10:06:27.138] [console] [info] epsilon=0.9707610086135772 is infeasible
[2022-01-28 10:06:37.594] [console] [info] Solver time 5.340476036071777
[2022-01-28 10:06:37.666] [console] [info] epsilon=0.29922117671584 is infeasible
[2022-01-28 10:06:50.545] [console] [info] Solver time 3.169828176498413
[2022-01-28 10:06:50.665] [console] [info] epsilon=-0.03654873923302862 is feasible
[2022-01-28 10:06:50.666] [console] [info] reset eps_min=0.0, eps_max=0.3357699159488686
[2022-01-28 10:07:00.667] [console] [info] Solver time 4.772485017776489
[2022-01-28 10:07:00.740] [console] [info] epsilon=0.1678849579744343 is infeasible
[2022-01-28 10:07:06.647] [console] [info] Solver time

[IRIS +  CERT]: Eps min -0.33868521538628993
[IRIS +  CERT]: Region found, vol =  0.43617411313252785 , halfaxis:  [3.98248604 1.65632853 1.45588804]


[2022-01-28 10:07:09.547] [console] [info] Solver time 0.7495331764221191
[2022-01-28 10:07:12.850] [console] [info] Solver time 1.3995280265808105
[2022-01-28 10:07:12.908] [console] [info] epsilon=2.330657392306855 is infeasible
[2022-01-28 10:07:19.008] [console] [info] Solver time 2.8988308906555176
[2022-01-28 10:07:19.106] [console] [info] epsilon=0.9959860884602825 is infeasible
[2022-01-28 10:07:28.942] [console] [info] Solver time 4.733088970184326
[2022-01-28 10:07:29.023] [console] [info] epsilon=0.32865043653699627 is infeasible
[2022-01-28 10:07:37.281] [console] [info] Solver time 4.008227109909058
[2022-01-28 10:07:37.378] [console] [info] epsilon=-0.005017389424646829 is infeasible
[2022-01-28 10:07:45.194] [console] [info] Solver time 1.7212021350860596
[2022-01-28 10:07:45.330] [console] [info] epsilon=-0.17185130240546836 is feasible
[2022-01-28 10:07:45.331] [console] [info] reset eps_min=0.0, eps_max=0.16683391298082162


In [ ]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [ ]:
#setup refine with python
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
iris_generator.regions = regions
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')